# Genetic Algorithms

Suppose that we have the following maximization problem:
$$
    \underset{x \in \mathbb{S}}{\operatorname{\argmax}} \; f(x)
$$
where $x$ must satisfy fixed constraints. 

Define $\mathbb{S}$ as the set of solutions, i.e. the $x$ that satisfy the costraints.
Assume that $\mathbb{S}$ is not empty and let $x_1, \dots, x_N$ be a set of sub-optimal. This subset is called **population** and each element of this subset is called **inidvidual**. As we did in single state methods, our goal is to improve our candidate solution.

The **genotype** of a solution $x$ is the representation of that solution (e.g. the data structure used to encode $x$, in OneMax is a string of bits).

The **phenotype** of a solution $x$ is a decoding of its genotype (e.g. the number written in base 10).

A genetic algorithm is an algorithm whose flow can be represented as follows:

<img src="img/ga_flow.png" alt="GA flow" width="600"/>

Generally, a genetic algorithm consists of:

- a **genetic representation** of the individuals;

- a **fitness function** that evaluates each individual;

- an **initialization** method, it generates a population of fixed size;

- a **selection** method, it replaces the individuals with lower fitness value with the ones with higher fitness, this method is based on the phenotype of a solution;

- a **crossover** method, it consists in merging the genotypes of two individuals in order to create two new individuals which will replace the old ones;

- **mutation** type, a mutation is a stochastic modification that happens to the genotype of an individual with a given probability,

- a **termination criterium**.



# GA for OneMax
Let's apply a GA to the OneMax problem.

In [1]:
import random

class OneMaxProblem:
    '''
        A simple class for the OneMax problem.
    '''
    def __init__(self, n):
        self.n = n

    def fitness(self, x):
        return len([digit for digit in x if digit == 1])
    
    def random_solution(self):
        return [random.randint(0, 1) for _ in range(self.n)]

## Population generation

Let's start from randomly generating a population $\mathbb{P} = \{x_1,\dots, x_N\}$

In [2]:
problem = OneMaxProblem(10)
N = 20 # population size
population = [problem.random_solution() for _ in range(N)]

## Selection methods

### Roulette wheel selection
The probability of an individual to be selected is proportional to its fitness:
$$
    p(x) = \frac{f(x)}{\sum_{y \in \mathbb{P}} f(y)}
$$

In [3]:
import random

def roulette_wheel_selection(population, fitness):
    '''
        Selects a solution from the population using the roulette wheel selection.
    '''
    total_fitness = sum([fitness(x) for x in population])
    probabilities = [fitness(x) / total_fitness for x in population]
    selected = random.choices(population, weights=probabilities, k=len(population))
    return selected

### Ranked selection 

The probability of an individual to be selected is inversely proprotional to its rank (w.r.t. their fitness values). Let 
$$
    rank(x) \in \{1 \dots N\}
$$

be the rank of the solution $x$. The probability of $x$ of being selected is 

$$
    p(x) = \frac{1}{N}\left( \gamma - (2 \gamma -2) \frac{rank(x)-1}{N-1} \right)
$$

where $1 \leq \gamma \leq 2$ is known as selection pressure.

In [4]:
def ranked_selection(population, fitness, gamma=1.5):
    '''
        Selects a solution from the population using the ranked selection.
    '''
    population.sort(key=lambda x: fitness(x), reverse=True)
    N = len(population)
    probabilities = [(gamma-(2*gamma-2)*(i-1)/(N-1))/N for i in range(N)]
    selected = random.choices(population, weights=probabilities, k=N)
    return selected

### Tournament selection

Let us define $t \in {1,2,\dots, N}$ as the tournament size. Now, we randomly extract $t$ individuals from the population and we select the individual with the best fitness. The selection pressure can be tuned via tournament size. A low tournament size implies a low selection pressure.

In [5]:
def tournament_selection(population, fitness, t = 5):
    '''
        Selects a solution from the population using the tournament selection.
    '''
    selected = []
    for _ in range(len(population)):
        tournament = random.choices(population, k=t)
        tournament.sort(key=lambda x: fitness(x), reverse=True)
        selected.append(tournament[0])
    return selected

## Crossover methods

### One-point crossover

Given two parent solutions, represented using bits, 
$$
    x = x^1 \ x^2 \ \dots x^n \\
    y = y^1 \ y^2 \dots y^n
$$
we select a crossover point $k \in {1,\dots,n}$ uniformly at random.
The offspring are
$$
    x_{new} = x^1 \dots x^k \ y^{k+1} \ \dots y^n \\
    y_{new} = y^1 \dots y^k \ x^{k+1} \ \dots x^n 
$$
A generalization of one-point crossover where more than one si selected. In particular $m$ points $k_1, \dots, k_m \in \{1, \dots, n\}$ are selected and the elements of the two parents are swapped between each paoinr of consecutive crossover points.

In [6]:
def onepoint_crossover(x, y):
    '''
        Performs a one-point crossover between x and y.
    '''
    n = len(x)
    c = random.randint(0, n-1)
    x_new = x[:c] + y[c:]
    y_new = y[:c] + x[c:]
    return x_new, y_new

def kpoints_crossover(x, y, k):
    '''
        Performs a k-points crossover between x and y.
    '''
    n = len(x)
    c = sorted(random.sample(range(1, n), k))
    for i in range(k):
        x_new = x[:c[i]] + y[c[i]:c[i+1]] + x[c[i+1]:]
        y_new = y[:c[i]] + x[c[i]:c[i+1]] + y[c[i+1]:]
    
    return x_new, y_new

### Uniform crossover

For each $i \in \{1,\dots,n\}$, with probability $0.5$ the $i^{th}$ element in the first offspring will be $x^i$, otherwise $y_i$. In the second offspring, the $i^{th}$ element is $y_i$, if $x^i$ was chosen in the first offspring and vice versa. 

In [7]:
def uniform_crossover(x, y):
    '''
        Performs a uniform crossover between x and y.
    '''
    n = len(x)
    x_new = [x[i] if random.random() < 0.5 else y[i] for i in range(n)]
    y_new = [y[i] if random.random() < 0.5 else x[i] for i in range(n)]
    return x_new, y_new


## Mutation
Given a solution
$$
    x = x^1 \ x^2 \ \dots x^n
$$
each bit $x^i$ is flipped with probability $p_{mut}$ (usually $p_{mut} = 1/n$).

In [8]:
def mutation(x, p):
    '''
        Performs a mutation on x with probability p.
    '''
    n = len(x)
    x_new = [x[i] if random.random() > p else 1-x[i] for i in range(n)]
    return x_new

## GA implementation

In [9]:
def best_individual(population, fitness):
    '''
        Returns the best individual from the population.
    '''
    return max(population, key=fitness)

In [10]:
def genetic_algorithm(starting_population, fitness, selection, crossover, mutation, p_c, p_m, max_iter=10):
    '''
        Runs the genetic algorithm.
    '''
    population = starting_population.copy()
    
    for iter in range(max_iter):
        print(f"Generation {iter}, best individual: {best_individual(population, fitness)} with fitness {fitness(best_individual(population, fitness))}")
        population = selection(population, fitness)
        for i in range(0, len(population), 2):
            if random.random() < p_c:
                population[i], population[i+1] = crossover(population[i], population[i+1])
        population = [mutation(x, p_m) for x in population]
    
    return best_individual(population, fitness)

In [11]:
solution = genetic_algorithm(population, problem.fitness, tournament_selection, onepoint_crossover, mutation, 0.8, 0.01)

Generation 0, best individual: [1, 0, 1, 1, 1, 1, 0, 1, 1, 1] with fitness 8
Generation 1, best individual: [1, 0, 1, 1, 1, 1, 0, 1, 1, 1] with fitness 8
Generation 2, best individual: [1, 1, 1, 0, 1, 1, 1, 1, 1, 1] with fitness 9
Generation 3, best individual: [1, 1, 1, 0, 1, 1, 1, 1, 1, 1] with fitness 9
Generation 4, best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] with fitness 10
Generation 5, best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] with fitness 10
Generation 6, best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] with fitness 10
Generation 7, best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] with fitness 10
Generation 8, best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] with fitness 10
Generation 9, best individual: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1] with fitness 10


In [12]:
solution

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]